In [16]:
import os
import sys
import yaml
import types
import pprint
import logging
import logging
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_ext.memory.canvas import TextCanvasMemory
from autogen_agentchat import EVENT_LOGGER_NAME, TRACE_LOGGER_NAME
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_core import SingleThreadedAgentRuntime
from autogen_ext.models.openai import OpenAIChatCompletionClient
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor


# Project import paths
# Get the current directory (which is project/test)
notebook_dir = os.getcwd()

# Compute absolute paths for lib and src (one level up from test/)
lib_path = os.path.abspath(os.path.join(notebook_dir, '..', 'lib'))
src_path = os.path.abspath(os.path.join(notebook_dir, '..', 'src'))

sys.path.append(lib_path)
sys.path.append(src_path)

from agents_creator import create_agents_from_config
from rag_helpers import rag_indexer_runner
from model_loader import load_model_client


In [2]:
logging.basicConfig(level=logging.WARNING)

# For trace logging.
trace_logger = logging.getLogger(TRACE_LOGGER_NAME)
trace_logger.addHandler(logging.StreamHandler())
trace_logger.setLevel(logging.DEBUG)

# For structured message logging, such as low-level messages between agents.
event_logger = logging.getLogger(EVENT_LOGGER_NAME)
event_logger.addHandler(logging.StreamHandler())
event_logger.setLevel(logging.DEBUG)

In [3]:
# Set up telemetry span exporter.
otel_exporter = OTLPSpanExporter(endpoint="http://localhost:4317", insecure=True)
span_processor = BatchSpanProcessor(otel_exporter)

# Set up telemetry trace provider.
tracer_provider = TracerProvider(resource=Resource({"service.name": "autogen-test-agentchat"}))
tracer_provider.add_span_processor(span_processor)
trace.set_tracer_provider(tracer_provider)

# Instrument the OpenAI Python library
OpenAIInstrumentor().instrument()

# we will get reference this tracer later using its service name
# tracer = trace.get_tracer("autogen-test-agentchat")

In [ ]:
rag_vector_memory = await rag_indexer_runner('/run/media/jun/Jun 6T/Code/TFM_BA_GPT/lib/rag_config.yaml')

/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All documents and URLs are already indexed.


ERROR:opentelemetry.exporter.otlp.proto.http.metric_exporter:Failed to export batch code: 404, reason: 404 page not found

ERROR:opentelemetry.exporter.otlp.proto.http.metric_exporter:Failed to export batch code: 404, reason: 404 page not found

ERROR:opentelemetry.exporter.otlp.proto.http.metric_exporter:Failed to export batch code: 404, reason: 404 page not found

ERROR:opentelemetry.exporter.otlp.proto.http.metric_exporter:Failed to export batch code: 404, reason: 404 page not found

ERROR:opentelemetry.exporter.otlp.proto.http.metric_exporter:Failed to export batch code: 404, reason: 404 page not found

ERROR:opentelemetry.exporter.otlp.proto.http.metric_exporter:Failed to export batch code: 404, reason: 404 page not found

ERROR:opentelemetry.exporter.otlp.proto.http.metric_exporter:Failed to export batch code: 404, reason: 404 page not found

ERROR:opentelemetry.exporter.otlp.proto.http.metric_exporter:Failed to export batch code: 404, reason: 404 page not found

ERROR:openteleme

In [5]:
# 1. Create the canvas memory and tool:
text_canvas_memory = TextCanvasMemory()
update_file_tool = text_canvas_memory.get_update_file_tool()

# 2. Register the tool as 'canvas.update_file_tool' for YAML import:
canvas = types.SimpleNamespace()
canvas.update_file_tool = update_file_tool
sys.modules['canvas'] = canvas  # This enables the tool import by your agent factory!

In [6]:
model_client = load_model_client("azure")

In [7]:
# Construct the absolute path to project/lib/agents_config.yaml
AGENTS_CONFIG = os.path.abspath(os.path.join(notebook_dir, '..', 'lib', 'agents_config.yaml'))
with open(AGENTS_CONFIG, "r") as f:
        config_yaml = yaml.safe_load(f)
agent_names = list(config_yaml.keys())

user_proxy, agents = await create_agents_from_config(
    AGENTS_CONFIG,
    agent_names=agent_names,
    model_client=model_client,
    vector_memory=rag_vector_memory,
    canvas_memory=text_canvas_memory,
    input_func=input
)

In [8]:
for name, agent in agents.items():
    print(f"\n--- Agent: {name} ---")
    pprint.pprint(vars(agent))




--- Agent: planning_agent ---
{'_description': 'An agent for planning tasks, this agent should be the first '
                 'to engage when given a new task.',
 '_handoff_tools': [],
 '_handoffs': {},
 '_is_running': False,
 '_max_tool_iterations': 1,
 '_memory': None,
 '_metadata': {},
 '_model_client': <autogen_ext.models.openai._openai_client.AzureOpenAIChatCompletionClient object at 0x7fb49cae17f0>,
 '_model_client_stream': False,
 '_model_context': <autogen_core.model_context._unbounded_chat_completion_context.UnboundedChatCompletionContext object at 0x7fb49c9da270>,
 '_name': 'planning_agent',
 '_output_content_type': None,
 '_output_content_type_format': None,
 '_reflect_on_tool_use': False,
 '_structured_message_factory': None,
 '_system_messages': [SystemMessage(content='You are a planning agent.\nYour job is to break down complex tasks into smaller, manageable subtasks.\nYour team members are:\n  - interviewer_agent\n  - summarizer_agent\n  - funct_spec_checker_agent\n  -

In [9]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

In [10]:
# Order agents (workflow order), user_proxy last
agent_order = [
    "planning_agent",
    "interviewer_agent",
    "summarizer_agent",
    "doc_writter_agent",
    "funct_spec_checker_agent",
    "diagram_creator_agent",
    "mermaid_code_reviewer_agent"
]
member_agents = [agents[name] for name in agent_order if name in agents] + [user_proxy]

In [11]:
termination = MaxMessageTermination(100000000) | TextMentionTermination("TERMINATE")

# --- Use SelectorGroupChat ---
team = SelectorGroupChat(
    member_agents,
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True
)

In [12]:
task = "Write the functional specifications for a new product"

In [17]:
runtime = SingleThreadedAgentRuntime(
            tracer_provider=trace.NoOpTracerProvider(),  # Disable telemetry for runtime.
        )
runtime.start()

In [18]:
await Console(team.run_stream(task=task))

await runtime.stop()

await model_client.close()

/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:270: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)
Model selected a valid name: planning_agent (attempt 1)
DEBUG:autogen_agentchat:Model selected a valid name: planning_agent (attempt 1)
Selected speaker: planning_agent
DEBUG:autogen_agentchat:Selected speaker: planning_agent
DEBUG:autogen_agentchat:Model selected a valid name: planning_agent (attempt 1)
Selected speaker: planning_agent
DEBUG:autogen_agentchat:Selected speaker: planning_agent


---------- TextMessage (user) ----------
Write the functional specifications for a new product
Write the functional specifications for a new product
---------- TextMessage (planning_agent) ----------
To write the functional specifications for a new product, we need to break down the task into manageable subtasks. Here's the plan:

1. **interviewer_agent**: Conduct interviews to understand the product requirements, stakeholders, and use cases.
2. **summarizer_agent**: Summarize the findings from the interviews into clear and concise product requirements.
3. **funct_spec_checker_agent**: Review the summarized requirements to ensure they are clear, actionable, and complete from a functional specification standpoint.
4. **diagram_creator_agent**: Create visual diagrams or workflows to represent the product's functionality and design based on the specifications.
5. **mermaid_code_reviewer_agent**: Review the generated diagrams or workflows and ensure the visual representation is accurate an

/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/agents/_assistant_agent.py:1118: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(
/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:270: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)
Model selected a valid name: interviewer_agent (attempt 1)
DEBUG:autogen_agentchat:Model selected a valid name: interviewer_agent (attempt 1)
Selecte

---------- MemoryQueryEvent (interviewer_agent) ----------
[MemoryContent(content='• \nFunctional Requirements Specification \n• \n1. Document Information  \no 1.1. Document Approvers & Reviewers \no 1.2. Document History \no 1.3. Reference Documents \no 1.4. Definitions, Acronyms and Abbreviations \n• \n2. Executive Summary  \no 2.1. Overview \no 2.2. Business Benefits \n• \n3. Scope  \no 3.1. In Scope \no 4.2. Out of Scope \no 3.3. Systems & Platforms LeanIX Meta-Data \n• \n4. Requirements  \no 4.1. Functional Requirements \no 4.2. Non-Functional Requirements \n• \n5. Functional Solution', mime_type='MemoryMimeType.TEXT', metadata={'chunk_index': 3, 'mime_type': 'MemoryMimeType.TEXT', 'source': 'Functional Requirements Specification Template.doc.pdf', 'score': 0.6303571164608002, 'id': '449a8a70-4043-4372-b176-0e7f32232870'}), MemoryContent(content='Functional Requirements Specification - \nTemplate \n \nFunctional Requirements Specification \nBS-[number] [title] \nDOCUMENTATION \nDo

/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:270: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)
Model selected a valid name: interviewer_agent (attempt 1)
DEBUG:autogen_agentchat:Model selected a valid name: interviewer_agent (attempt 1)
Selected speaker: interviewer_agent
DEBUG:autogen_agentchat:Selected speaker: interviewer_agent
DEBUG:autogen_agentchat:Model selected a valid name: interviewer_agent (attempt 1)
Selected speaker: interviewer_agent
DEBUG:autogen_agentchat:Selected speaker: interviewer_agent


...

---------- ModelClientStreamingChunkEvent (interviewer_agent) ----------
What travelWhat travel package types or themes does the bot prioritize (e.g., international tours, staycations, adventure package types or themes does the bot prioritize (e.g., international tours, staycations, adventure holidays, luxury travel)?
 holidays, luxury travel)?


/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:270: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)
Model selected a valid name: interviewer_agent (attempt 1)
DEBUG:autogen_agentchat:Model selected a valid name: interviewer_agent (attempt 1)
Selected speaker: interviewer_agent
DEBUG:autogen_agentchat:Selected speaker: interviewer_agent
DEBUG:autogen_agentchat:Model selected a valid name: interviewer_agent (attempt 1)
Selected speaker: interviewer_agent
DEBUG:autogen_agentchat:Selected speaker: interviewer_agent


---------- ModelClientStreamingChunkEvent (interviewer_agent) ----------
LetLet’s move forward to the’s move forward to the decision-making logic of the bot.

### Question 5:  
What specific "cool decision-making logic of the bot.

### Question 5:  
What specific "cool things" or features about the things" or features about the travel packages should the bot highlight when trying to convince the user?
 travel packages should the bot highlight when trying to convince the user?


ERROR:autogen_core:Error processing publish message for SelectorGroupChatManager_a22ea84d-f1bc-4569-bfde-d94ace401c08/a22ea84d-f1bc-4569-bfde-d94ace401c08
Traceback (most recent call last):
  File "/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
    )
    ^
  File "/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_core/_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/run/media/jun/Jun 6T/Co

RuntimeError: InternalServerError: Error code: 500 - {'error': {'message': 'The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently. For more information, please see https://aka.ms/model-error', 'type': 'model_error', 'param': None, 'code': None}}
Traceback:
Traceback (most recent call last):

  File "/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_base_group_chat_manager.py", line 157, in handle_agent_response
    await self._transition_to_next_speakers(ctx.cancellation_token)

  File "/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_base_group_chat_manager.py", line 169, in _transition_to_next_speakers
    speaker_names = await speaker_names_future
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_se